In [ ]:
import sys
import os

# Add the path to the directory where your module is located
module_path = os.path.abspath(os.path.join('..', 'C:\\Users\\pier1\\OneDrive\\Desktop\\uni\\Master\\2.Semester\\Machine Learning (WIWI)\\Project\\Data for depression\\'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Now you can import your module
from helpers import *

In [ ]:
import pandas as pd
import numpy as np
from itertools import combinations
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from NN import *

In [ ]:
df = pd.read_csv('..\data\Threshold_10_Operator_+_Depressionfeature_MH_PHQ_S_PercentofDataset_100.csv')
#print_information(df)
# 0.9 train, 0.1 test
train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)
# 0.8 train, 0.2 validation
train_df, validation_df = train_test_split(train_df, train_size=0.8, random_state=42)

train_features, train_targets = df_to_tensor(train_df,features_column='FEATURES', target_col='Depression')
validation_features, validation_targets = df_to_tensor(validation_df, features_column='FEATURES', target_col='Depression')
test_features, test_targets = df_to_tensor(test_df, features_column='FEATURES', target_col='Depression')

print(train_features.shape, train_targets.shape, validation_features.shape, validation_targets.shape, test_features.shape, test_targets.shape)

train_dataset = CustomDataset(train_features, train_targets)
validation_dataset = CustomDataset(validation_features, validation_targets)
test_dataset = CustomDataset(test_features, test_targets)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=32, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
# Configuration
input_size = 56
hidden_size = 128
num_epochs = 30
learning_rate_values = [0.001,0.002,0.004,0.005,0.01]
momentum_values = [0.9,0.95,0.98]
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize the model, criterion, and optimizer
#model = Depression_Classifier_v_0(input_size, hidden_size).to(device)
results = {}
#criterion = nn.BCEWithLogitsLoss()  # Combines sigmoid and binary cross-entropy loss
criterion = nn.BCELoss()
optimizers = ['Adam', 'SGD']

for i in range(3):
    for optimizer in optimizers:
        if optimizer == 'Adam':
            for learning_rate in learning_rate_values:
                if i == 0:
                    m = Depression_Classifier_v_0(input_size, hidden_size).to(device)
                elif i == 1:
                    m = Depression_Classifier_v_1(input_size, hidden_size).to(device)
                else:
                    m = Depression_Classifier_v_2(input_size, hidden_size).to(device)

                optimizer = optim.Adam(m.parameters(), lr=learning_rate)

                train_model_cross_validation(m, train_dataloader,validation_dataloader,criterion, optimizer, num_epochs, device)

                validation_loss, accuracy = evaluate_model(m, test_dataloader, criterion, device)

                # Store results
                results[(f'Model_{i}', optimizer, learning_rate)] = {
                    'validation_loss': validation_loss,
                    'accuracy': accuracy
                }
        else:
            for momentum, learning_rate in itertools.product(momentum_values, learning_rate_values):
                if i == 0:
                    m = Depression_Classifier_v_0(input_size, hidden_size).to(device)
                elif i == 1:
                    m = Depression_Classifier_v_1(input_size, hidden_size).to(device)
                else:
                    m = Depression_Classifier_v_2(input_size, hidden_size).to(device)
                    
                optimizer = optim.SGD(m.parameters(), lr=learning_rate, momentum=momentum)

                train_model_cross_validation(m, train_dataloader,validation_dataloader,criterion, optimizer, num_epochs, device)

                validation_loss, accuracy = evaluate_model(m, test_dataloader, criterion, device)

                # Store results
                results[(f'Model_{i}', optimizer, learning_rate,momentum)] = {
                    'validation_loss': validation_loss,
                    'accuracy': accuracy
                }
                